In [1]:
# mobilenet_transfer_train.py
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ------- User settings -------
base_dir = r'C:\Users\fab\Documents\projects\smart-banana-expo\smart-banana\BananaLSD\AugmentedSet'  # update if needed
diseases = ['cordana', 'healthy', 'pestalotiopsis', 'sigatoka']
img_height = 160   # smaller than 224 to save compute/size
img_width = 160
batch_size = 32
epochs_stage1 = 8     # initial training with frozen base
epochs_stage2 = 8     # fine-tuning after unfreeze
learning_rate = 1e-4
model_out_dir = 'saved_models'
os.makedirs(model_out_dir, exist_ok=True)
# -----------------------------

# show counts
for d in diseases:
    p = os.path.join(base_dir, d)
    n = len(os.listdir(p)) if os.path.exists(p) else 0
    print(f"{d}: {n} images")

# Data generators with preprocessing for MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = len(diseases)

# Build MobileNetV2-based model
from tensorflow.keras.applications import MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False,
                         input_shape=(img_height, img_width, 3))
base_model.trainable = False  # freeze the pretrained backbone

inputs = layers.Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)   # small head
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


cordana: 400 images
healthy: 400 images
pestalotiopsis: 400 images
sigatoka: 400 images
Found 1280 images belonging to 4 classes.
Found 320 images belonging to 4 classes.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,468 (9.24 MB)

 Trainable params: 164,484 (642.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [3]:
# Save only weights
checkpoint_path = os.path.join(model_out_dir, 'best_mobilenetv2.weights.h5')  # <- fix here
callbacks = [
    ModelCheckpoint(
        checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1
    )
]


# Stage 1: train head (backbone frozen)
history1 = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs_stage1,
    callbacks=callbacks
)

if os.path.exists(checkpoint_path):
    model.load_weights(checkpoint_path)   # load weights into same architecture


# Stage 2: fine-tune - unfreeze last few layers of backbone
base_model.trainable = True

# Freeze all layers except last N blocks to avoid big changes
# You can experiment with unfreezing more/less
fine_tune_at = 100  # layer index to start fine-tuning (tweak if needed)

for i, layer in enumerate(base_model.layers):
    layer.trainable = i >= fine_tune_at

model.compile(
    optimizer=Adam(learning_rate=learning_rate/10),  # lower LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs_stage2,
    callbacks=callbacks
)


Epoch 1/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3400 - loss: 1.7243
Epoch 1: val_accuracy improved from None to 0.64062, saving model to saved_models\best_mobilenetv2.weights.h5
40/40 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.4141 - loss: 1.4745 - val_accuracy: 0.6406 - val_loss: 0.8494 - learning_rate: 1.0000e-04
Epoch 2/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 591ms/step - accuracy: 0.6075 - loss: 0.9621
Epoch 2: val_accuracy improved from 0.64062 to 0.80937, saving model to saved_models\best_mobilenetv2.weights.h5
40/40 ━━━━━━━━━━━━━━━━━━━━ 31s 768ms/step - accuracy: 0.6578 - loss: 0.8556 - val_accuracy: 0.8094 - val_loss: 0.5213 - learning_rate: 1.0000e-04
Epoch 3/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 629ms/step - accuracy: 0.7216 - loss: 0.6731
Epoch 3: val_accuracy improved from 0.80937 to 0.86250, saving model to saved_models\best_mobilenetv2.weights.h5
40/40 ━━━━━━━━━━━━━━━━━━━━ 34s 851ms/step - accuracy: 0.7453 - loss: 0.6317 - val_accuracy: 0.8625 - val_loss: 0.4165 - lea

In [4]:
final_keras_path = os.path.join(model_out_dir, 'banana_mobilenetv2_final.keras')
model.save(final_keras_path)


In [5]:

# Final evaluation
loss, acc = model.evaluate(validation_generator)
print(f"Final validation accuracy: {acc*100:.2f}%")

# Save final Keras model (SavedModel format or .keras)
final_keras_path = os.path.join(model_out_dir, 'banana_mobilenetv2_final.keras')
model.save(final_keras_path)
print(f"Keras model saved to: {final_keras_path}")

# Save class indices for inference later
import json
class_indices_path = os.path.join(model_out_dir, 'class_indices.json')
with open(class_indices_path, 'w') as f:
    json.dump(train_generator.class_indices, f)
print(f"Saved class indices to: {class_indices_path}")

# Show file size
def sizeof(path):
    size = os.path.getsize(path) / (1024*1024)
    return f"{size:.2f} MB"

print("Keras model size:", sizeof(final_keras_path))


10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 318ms/step - accuracy: 0.9062 - loss: 0.2420
Final validation accuracy: 90.62%
Keras model saved to: saved_models\banana_mobilenetv2_final.keras
Saved class indices to: saved_models\class_indices.json
Keras model size: 25.30 MB


In [6]:

# -----------------------------
# Convert to TFLite with post-training quantization (smaller file)
# -----------------------------
# (1) Basic dynamic range quantization
# Load the Keras model file
model = tf.keras.models.load_model(final_keras_path)

# Convert directly from the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
tflite_path = os.path.join(model_out_dir, 'banana_mobilenetv2_dynamic_quant.tflite')
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)
print("Saved TFLite (dynamic quant) to:", tflite_path)
print("TFLite size:", sizeof(tflite_path))

# (2) Full integer quantization (requires a calibration dataset generator)
def representative_data_gen():
    for i in range(100):  # 100 calibration samples
        img, _ = next(train_generator)
        yield [img.astype(np.float32)]

# Load the Keras model file
model = tf.keras.models.load_model(final_keras_path)

# Convert directly from the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# to ensure integer ops:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8  # or tf.int8
converter.inference_output_type = tf.uint8
try:
    tflite_int8 = converter.convert()
    tflite_int8_path = os.path.join(model_out_dir, 'banana_mobilenetv2_int8.tflite')
    with open(tflite_int8_path, 'wb') as f:
        f.write(tflite_int8)
    print("Saved TFLite (int8) to:", tflite_int8_path)
    print("TFLite int8 size:", sizeof(tflite_int8_path))
except Exception as e:
    print("Integer quantization failed (may not be supported on all ops/devices):", e)

# (optional) Float16 quantization (good tradeoff)
# Load the Keras model file
model = tf.keras.models.load_model(final_keras_path)

# Convert directly from the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_fp16 = converter.convert()
tflite_fp16_path = os.path.join(model_out_dir, 'banana_mobilenetv2_fp16.tflite')
with open(tflite_fp16_path, 'wb') as f:
    f.write(tflite_fp16)
print("Saved TFLite (float16) to:", tflite_fp16_path)
print("TFLite fp16 size:", sizeof(tflite_fp16_path))

# Quick test: run one prediction with TFLite dynamic quant model
import numpy as np
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# read one sample from validation set
x_val, y_val = next(validation_generator)
sample = x_val[0:1]  # single image
# If model expects float input (dynamic quant uses float), set directly
interpreter.set_tensor(input_details[0]['index'], sample.astype(np.float32))
interpreter.invoke()
pred = interpreter.get_tensor(output_details[0]['index'])
pred_class = np.argmax(pred, axis=-1)[0]
print("TFLite predicted class index:", pred_class)
print("Mapping:", train_generator.class_indices)


INFO:tensorflow:Assets written to: C:\Users\fab\AppData\Local\Temp\tmpnqmvwqn4\assets


INFO:tensorflow:Assets written to: C:\Users\fab\AppData\Local\Temp\tmpnqmvwqn4\assets


Saved TFLite (dynamic quant) to: saved_models\banana_mobilenetv2_dynamic_quant.tflite


NameError: name 'sizeof' is not defined